# NER with GoLLIE-TF

### Import requeriments

See the requeriments.txt file in the main directory to install the required dependencies

In [2]:
import sys

sys.path.append("../")  # Add the GoLLIE base directory to sys path

In [ ]:
import rich
import logging
from src.model.load_model import load_model
import black
import inspect
from jinja2 import Template
import tempfile
from src.tasks.utils_typing import AnnotationList

logging.basicConfig(level=logging.INFO)
from typing import Dict, List, Type

## Load GoLLIE

We will load GOLLIE-7B from the huggingface-hub.
You can use the function AutoModelForCausalLM.from_pretrained if you prefer it. However, we provide a handy load_model function with many functionalities already implemented that will assist you in reproducing our results.

Please note that setting use_flash_attention=True is mandatory. Our flash attention implementation has small numerical differences compared to the attention implementation in Huggingface. Using use_flash_attention=False will result in the model producing inferior results. Flash attention requires an available CUDA GPU. Running GOLLIE pre-trained models on a CPU is not supported. We plan to address this in future releases.

- Set force_auto_device_map=True to automatically load the model on available GPUs.
- Set quantization=4 if the model doesn't fit in your GPU memory.

In [1]:
import torch
print(torch.cuda.is_available())

False


In [ ]:
model, tokenizer = load_model(
    inference=True,
    model_weights_name_or_path="ychenNLP/GoLLIE-7B-TF",
    quantization=None,
    use_lora=False,
    force_auto_device_map=True,
    use_flash_attention=True,
    torch_dtype="bfloat16",
)

In [ ]:
test_input = r'''# The following lines describe the task definition
@dataclass
class LLM(Entity):
    """Large language model names or model names. This is used for deep learning and NLP tasks."""

    span: str  # Such as: "GPT-3.5", "LLama=7B", "ChatGPT"

@dataclass
class Hyperparams(Entity):
    """Hyperparameter used for training large language  models. Including learning rate, scheduler, architecture"""

    span: str  # Such as: "layernorm", "cosine scheduler"

# This is the text to analyze
text = "GoLLIE-7B-TFが本日リリースされました！ 1つのNVIDIA A100 GPUで推論が可能なサイズです 学習率は1e-4です 訓練にはLoRAが使用されています"

# This is the English translation of the text
eng_text = "GoLLIE-7B-TF is released today! * Sized for inference on 1 NVIDIA A100 GPUs * learning rate 1e-4 * LoRA is used for training"

# Using translation and fusion
# (1) generate annotation for eng_text
# (2) generate annotation for text

# The annotation instances that take place in the eng_text above are listed here
result = [
'''

In [ ]:
model_input = tokenizer(test_input, return_tensors="pt")

print(model_input["input_ids"])

model_input["input_ids"] = model_input["input_ids"][:, :-1]
model_input["attention_mask"] = model_input["attention_mask"][:, :-1]

model_ouput = model.generate(
    **model_input.to(model.device),
    max_new_tokens=128,
    do_sample=False,
    min_new_tokens=0,
    num_beams=1,
    num_return_sequences=1,
)
print(tokenizer.batch_decode(model_ouput))

'''
['<s> # The following lines describe the task definition\n@dataclass\nclass LLM(Entity):\n    """Large language model names or model names. This is used for deep learning and NLP tasks."""\n\n    span: str  # Such as: "GPT-3.5", "LLama=7B", "ChatGPT"\n\n@dataclass\nclass Hyperparams(Entity):\n    """Hyperparameter used for training large language  models. Including learning rate, scheduler, architecture"""\n\n    span: str  # Such as: "layernorm", "cosine scheduler"\n\n# This is the text to analyze\ntext = "GoLLIE-7B-TFが本日リリースされました！ 1つのNVIDIA A100 GPUで推論が可能なサイズです 学習率は1e-4です 訓練にはLoRAが使用されています"\n\n# This is the English translation of the text\neng_text = "GoLLIE-7B-TF is released today! * Sized for inference on 1 NVIDIA A100 GPUs * learning rate 1e-4 * LoRA is used for training"\n\n# Using translation and fusion\n# (1) generate annotation for eng_text\n# (2) generate annotation for text\n\n# The annotation instances that take place in the eng_text above are listed here\nresult = [\n    LLM(span="GoLLIE-7B-TF"),\n    Hyperparams(span="learning rate 1e-4"),\n    Hyperparams(span="LoRA"),\n]\n\n# The annotation instances that take place in the text above are listed here\nfinal_result = [\n    LLM(span="GoLLIE-7B-TF"),\n    Hyperparams(span="学習率は1e-4"),\n    Hyperparams(span="LoRA"),\n]\n</s>']
'''